In [ ]:
!pip install -q pyomo

     |████████████████████████████████| 9.0MB 1.5MB/s 
     |████████████████████████████████| 256kB 47.3MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 
     |████████████████████████████████| 163kB 48.2MB/s 


In [ ]:
from pyomo.environ import * 

In [ ]:
import numpy as np

In [ ]:
coef = np.loadtxt('Lab4_Ex1_coef.txt', delimiter=',')

In [ ]:
print(coef.shape)
print('Num rows:',coef.shape[0])
print('Num cols:',coef.shape[1])

(6, 7)
Num rows: 6
Num cols: 7


In [ ]:
print(coef)

[[-0.5  -1.25 -5.    3.   10.   25.    0.  ]
 [ 0.8  -1.3   0.   -1.    0.    0.   12.  ]
 [ 0.    1.   -1.    0.   -1.    0.    0.6 ]
 [ 0.    0.    1.    0.    0.   -1.    1.2 ]
 [ 0.5   0.8   4.    0.    0.    0.   45.  ]
 [ 0.9   1.5   0.    0.    0.    0.   27.  ]]


In [ ]:
model = ConcreteModel()

In [ ]:
N = coef.shape[1]-1
M = coef.shape[0]-1

In [ ]:
#coefficients for objective function
obj_coef = coef[0,:-1]
print(obj_coef.shape)
print(obj_coef)

(6,)
[-0.5  -1.25 -5.    3.   10.   25.  ]


In [ ]:
constr_coef = coef[1:,:-1]
print(constr_coef.shape)
print(constr_coef)

(5, 6)
[[ 0.8 -1.3  0.  -1.   0.   0. ]
 [ 0.   1.  -1.   0.  -1.   0. ]
 [ 0.   0.   1.   0.   0.  -1. ]
 [ 0.5  0.8  4.   0.   0.   0. ]
 [ 0.9  1.5  0.   0.   0.   0. ]]


In [ ]:
constr_rhs = coef[1:,-1]
print(constr_rhs.shape)
print(constr_rhs)

(5,)
[12.   0.6  1.2 45.  27. ]


In [ ]:
lower_bound = 0
upper_bound = np.inf

In [ ]:
col_indices = np.arange(N)
row_indices = np.arange(M)
print(col_indices)
print(row_indices)


[0 1 2 3 4 5]
[0 1 2 3 4]


In [ ]:
model.x = Var(col_indices)

In [ ]:
model.constraints = ConstraintList()

In [ ]:
for i in row_indices:
  model.constraints.add(summation(constr_coef[i],model.x) <= constr_rhs[i])

In [ ]:
for j in col_indices:
  model.x[j].setlb(lower_bound)
  model.x[j].setub(upper_bound)

In [ ]:
model.objective = Objective(expr = summation(obj_coef,model.x), sense=maximize)

In [ ]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :   inf : False :  True :  Reals
          1 :     0 :  None :   inf : False :  True :  Reals
          2 :     0 :  None :   inf : False :  True :  Reals
          3 :     0 :  None :   inf : False :  True :  Reals
          4 :     0 :  None :   inf : False :  True :  Reals
          5 :     0 :  None :   inf : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : - 0.5*x[0] - 1.25*x[1] - 5.0*x[2

In [ ]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 144579 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Processing 

In [ ]:
opt_cbc = SolverFactory('cbc')

In [ ]:
result = opt_cbc.solve(model)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

    model=unknown;
        message from solver=<undefined>

Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: None
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 7
  Number of nonzeros: 6
  Sense: maximize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: unbounded
  Termination message: Model was proven to be unbounded.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.021439790725708008

Solver status: warning
Solver termination condition: unbounded


7.Answer>>>  Solver termination condition: unbounded
(means that the objective function takes either +ve infinity or -ve infinity value).Here it is -ve infinity.It is unbounded.So we need to change the sense of the objective function.

In [ ]:
#changing the sense of the objective function
model.objective.set_sense(minimize)

In [ ]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :   inf : False :  True :  Reals
          1 :     0 :  None :   inf : False :  True :  Reals
          2 :     0 :  None :   inf : False :  True :  Reals
          3 :     0 :  None :   inf : False :  True :  Reals
          4 :     0 :  None :   inf : False :  True :  Reals
          5 :     0 :  None :   inf : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : - 0.5*x[0] - 1.25*x[1] - 5.0*x[2

In [ ]:
result = opt_cbc.solve(model)
print(result)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: -17.2125
  Upper bound: -17.2125
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 7
  Number of nonzeros: 6
  Sense: minimize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 4
  Error rc: 0
  Time: 0.022028446197509766
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal


9.Answer>>> After changing the sense of the objective function we arrived at the optimal solution.



In [ ]:
# display solution
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model.x[i].value)
print('\n Constraints')
model.constraints.display()


Objective =  -17.2125

Decision Variables
x[ 0 ] =  17.925
x[ 1 ] =  1.8
x[ 2 ] =  1.2
x[ 3 ] =  0.0
x[ 4 ] =  0.0
x[ 5 ] =  0.0

 Constraints
constraints : Size=5
    Key : Lower : Body               : Upper
      1 :  None : 12.000000000000002 :  12.0
      2 :  None : 0.6000000000000001 :   0.6
      3 :  None :                1.2 :   1.2
      4 :  None :            15.2025 :  45.0
      5 :  None :            18.8325 :  27.0
